***
## Модели, связанные «один к одному»

Продолжаем развивать проект ACME: создадим модель для оригинальных названий фильмов; назовём её `OriginalTitle` и свяжем с моделью `VideoProduct` «один к одному».

При описании связанных моделей важен порядок, в котором они размещены в коде: синтаксис описания связи отличается в зависимости от того, какая модель описана первой, а какая — второй.

Модель `VideoProduct` ссылается на модель `OriginalTitle`. Чтобы модели `VideoProduct` было на что сослаться — сперва опишем модель `OriginalTitle`, а ниже — `VideoProduct`:

In [ ]:
# cinema/models.py
from django.db import models

class OriginalTitle(models.Model):
    title = models.CharField(max_length=128)

class VideoProduct(models.Model):
    title = models.CharField(max_length=128)
    # Описываем поле, ссылающееся на модель OriginalTitle:
    original_title = models.OneToOneField(
        # На какую модель ссылаемся
        OriginalTitle,
        # Поведение при удалении:
        # если оригинальное имя будет удалено,
        # то и сам фильм будет удалён. 
        on_delete=models.CASCADE
    )

***
## Ссылающуюся модель можно описать выше той, на которую она ссылается.

В этом случае имя модели, на которую ведёт ссылка, нужно указать в кавычках:

In [ ]:
# cinema/models.py
from django.db import models

class VideoProduct(models.Model):
    title = models.CharField(max_length=128)
    original_title = models.OneToOneField(
        'OriginalTitle',  # Имя связанной модели заключаем в кавычки.
        on_delete=models.CASCADE
    )

# Модель OriginalTitle описана позже, чем VideoProduct.
class OriginalTitle(models.Model):
    title = models.CharField(max_length=128)

***
При миграции Django ORM создаст такие запросы:

```sql
CREATE TABLE "cinema_originaltitle" 
("id" integer NOT NULL PRIMARY KEY AUTOINCREMENT,
"title" varchar(128) NOT NULL
);

CREATE TABLE "cinema_videoproduct" 
("id" integer NOT NULL PRIMARY KEY AUTOINCREMENT, 
"title" varchar(128) NOT NULL, 
"original_title_id" bigint NOT NULL UNIQUE REFERENCES "cinema_originaltitle" ("id")
);
```

Чудеса с именами: имя поля `original_title` Django ORM превратил в `original_title_id`, приклеив к нему суффикс `_id`. 

Переведём SQL-запрос на русский язык:

1. Создай таблицу `cinema_originaltitle` с такими полями:

* `id` — автоинкрементный первичный ключ типа `integer`. При добавлении каждой новой записи значение в этом поле будет автоматически увеличиваться на единицу. Django ORM автоматически создаёт такое поле для каждой модели.

* `title` — обязательное поле (`NOT NULL`) типа «строка»; длина строки — не более 128 символов.

2. Создай таблицу `cinema_videoproduct` с такими полями:

* `id` — автоинкрементный первичный ключ типа `integer`.

* `title` — обязательное поле (`NOT NULL`), строка не больше 128 символов;

* `original_title_id` — внешний ключ типа `bigint` (большое целое число).

    Тип `bigint` похож на тип `integer`; разница в том, что `integer` не может принимать значения более 2147483647, а у `bigint` этого ограничения нет. При большом количестве записей значение id может и преодолеть лимит, установленный для `integer`.

    Поле `original_title_id` обязательное (не может быть `NULL`), с уникальным (`UNIQUE`) в пределах таблицы значением; в параметре `REFERENCES "cinema_originaltitle" ("id")` описано, что это поле ссылается на колонку `id` таблицы `cinema_originaltitle`.

После наполнения данными таблицы, созданные Django ORM, будут выглядеть примерно так:

![alt text](https://pictures.s3.yandex.net/resources/image_1707808940.png)

***
## Обязательный аргумент для ссылающихся полей: on_delete

Инструкция `on_delete=models.CASCADE` в модели `VideoProduct` устанавливает такие правила: если удаляется запись из `cinema_originaltitle` — будет удалена и ссылающаяся на неё запись из таблицы `cinema_videoproduct`.

Есть и другой вариант: при удалении оригинального названия из таблицы `cinema_originaltitle` запись в таблице `cinema_videoproduct` можно сохранить; но для этого нужно сделать так, чтобы запись из `cinema_videoproduct` никуда не ссылалась (оригинального названия-то нет); в поле `original_title_id` этой записи нужно установить `NULL`.

Для этого в описании поля модели нужно задать параметр `on_delete=models.SET_NULL`. При удалении объекта, на который ведёт ссылка, в ссылающихся записях вместо ссылки на объект будет установлен `NULL`. Чтобы провернуть этот фокус с `SET_NULL` — поле `original_title_id` должно принимать значение `NULL`; для этого в описании поля модели указывают аргумент `null=True`:

In [ ]:
class VideoProduct(models.Model):
    ...
    original_title = models.OneToOneField(
        OriginalTitle,  
        on_delete=models.SET_NULL,
        null=True
    )

Для `on_delete` есть и другие аргументы:  `SET_DEFAULT`, `PROTECT` , `RESTRICT` и `DO_NOTHING`. 

***
## Модели, связанные «многие к одному»

Продолжаем строить базу данных: создадим модель `ProductType`, которая будет описывать тип фильма. Связь между моделями `VideoProduct` и `ProductType` должна быть «многие к одному»: много фильмов могут относиться к одному типу.

![alt text](https://pictures.s3.yandex.net/resources/image_1707808954.png)

Сначала опишем модель `ProductType`, а следом — `VideoProduct`, которая ссылается на `ProductType`. 

Связь «многие к одному» указывается  с помощью класса `ForeignKey`:

In [ ]:
# cinema/models.py
from django.db import models

class ProductType(models.Model):
    title = models.CharField(max_length=128)

class VideoProduct(models.Model):
    title = models.CharField(max_length=128)
    product_type = models.ForeignKey(
        ProductType,
        on_delete=models.CASCADE
    )

***
## Cвязь N:M «на коленке»

Оригинальные значения и типы фильмов подключили, осталось разобраться с режиссёрами. Создадим таблицу и для них.

У фильма может быть один режиссёр, а может быть целая команда, и всех их нужно связать с фильмом. Однако каждый режиссёр может быть автором нескольких фильмов.

Получается, записи таблиц фильмов и режиссёров должны быть связаны как «многие ко многим».

Такую связь можно создать через вспомогательную промежуточную таблицу:

![alt text](https://pictures.s3.yandex.net/resources/S02_236_1679038813.png)

Создадим модель `Director` (в ней будем хранить информацию о режиссёрах) и модель для промежуточной таблицы `—DirectorVideoProduct`: в каждой записи этой таблицы будут сопоставлены `id` фильма и `id` связанного с ним режиссёра.

In [ ]:
from django.db import models

class VideoProduct(models.Model):
    title = models.CharField(max_length=128)

class Director(models.Model):
    full_name = models.CharField(max_length=128)

class DirectorVideoProduct(models.Model):
    video_product = models.ForeignKey(VideoProduct, on_delete=models.CASCADE)
    director = models.ForeignKey(Director, on_delete=models.CASCADE)

***
## Связь N:M в Django ORM

В Django ORM для создания связи «многие ко многим» предусмотрен тип поля `ManyToManyField`:

In [ ]:
from django.db import models

class Director(models.Model):
    full_name = models.CharField(max_length=128)

class VideoProduct(models.Model):
    title = models.CharField(max_length=128)
    directors = models.ManyToManyField(Director)  # Вот оно, поле! 

Увидев инструкцию `models.ManyToManyField`, Django ORM поймёт, что без промежуточной таблицы не обойтись — и сам создаст её, не заставляя разработчика загромождать код вспомогательными моделями. 

SQL-запрос будет выглядеть почти так же, как и в предыдущем примере, где промежуточная таблица была описана вручную. Единственная разница — Django ORM создаст индекс по составному первичному ключу `'videoproduct_id'`, `'director_id'`.

В большинстве случаев стандартное поле `ManyToManyField` — это всё, что нужно для связи «многие ко многим». Но иногда в промежуточной модели требуется создать дополнительное поле, например, чтобы охарактеризовать связь между двумя моделями. В нашем примере в таком поле можно хранить информацию о дате начала работы конкретного режиссёра над конкретной картиной или причину, по которой для работы над фильмом выбрали именно этого режиссёра.

При использовании поля `ManyToManyField` Django ORM позволяет указать промежуточную модель, которая будет использоваться для управления отношениями «многие ко многим». В этой модели можно описать и дополнительные поля (например, добавить поле для текста, описывающего эту связь). 

Промежуточная модель указывается при описании поля `ManyToManyField` при помощи параметра `through`. Для нашего кинематографического примера код будет выглядеть так:



In [ ]:
from django.db import models

class Director(models.Model):
    full_name = models.CharField(max_length=128)

class VideoProduct(models.Model):
    title = models.CharField(max_length=128)
    # Параметр through указывает, какую модель надо назначить промежуточной:
    directors = models.ManyToManyField(Director, through='Partnership')

class Partnership(models.Model):
    # Поле, ссылающееся на модель Director:
    director = models.ForeignKey(Director, on_delete=models.CASCADE)
    # Поле, ссылающееся на модель VideoProduct:
    videoproduct = models.ForeignKey(VideoProduct, on_delete=models.CASCADE)
    # Дополнительные поля:
    # дата начала работы режиссёра над фильмом...
    date_joined = models.DateField()
    # ...и история о том, почему на фильм пригласили именно этого режиссёра.
    invite_reason = models.CharField(max_length=300)

В описании модели-посредника необходимо явно указать внешние ключи для моделей, связанных «многие ко многим» (в нашем примере это поля `director` и `videoproduct`). Это явное объявление и определяет, как связаны две модели.

При создании связи N:M разработчики в большинстве случаев применяют поле `ManyToManyField` (с параметром `through` или без него); при использовании этого поля Django ORM заметно упрощает работу со связанными моделями, предоставляя разработчику множество дополнительных инструментов, которых не получить, создав связь «на коленке». 

***
## Хозяйке на заметку

* О том, как работают значения `SET_DEFAULT`, `PROTECT` , `RESTRICT` и `DO_NOTHING` для `on_delete`, можно узнать [в документации](https://docs.djangoproject.com/en/3.2/ref/models/fields/#django.db.models.CASCADE).

***
## Имя для обратной связи между таблицами

Если в модели описаны поля, для которых указан тип `ForeignKey`, то объекты связанной модели получают специальный интерфейс, через который можно получить доступ к объектам исходной модели. Для этого используется опциональный параметр `related_name`. 

Этот параметр позволяет задать имя атрибута, используемого для обратной связи от связанной модели к модели, которая определяет отношение.

Вот пример использования `related_name`:

In [ ]:
from django.db import models

class Author(models.Model):
    name = models.CharField(max_length=100)

class Book(models.Model):
    title = models.CharField(max_length=100)
    author = models.ForeignKey(Author, on_delete=models.CASCADE, related_name='books')

В этом примере, у модели `Book` есть поле `author`, связанное с моделью `Author` через `ForeignKey`. Параметр `related_name='books'` указывает, что для доступа ко всем книгам автора, связанным с объектом модели `Author`, можно использовать атрибут `books`. Например вот так:

```py
author = Author.objects.get(name='Александр Пушкин')
pushkin_books = author.books.all() 
```

Если не указывать `related_name`, то для доступа к связанным книгам пришлось бы использовать имя `book_set`. Такое имя генерируется автоматически добавлением постфикса «_set» к имени модели.

```py
pushkin_books = author.book_set.all() 
```

Использование `related_name` удобно в случаях, когда:

* необходимо предоставить более понятное и читаемое имя для доступа к связанным объектам вместо использования автоматически сгенерированных имен.

* в случае, когда у модели есть несколько полей, связанных с другой моделью и `related_name` помогает различать имена обратных связей и предотвращает возможные конфликты имен.